# Analysis with the rest feature

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
df=pd.read_csv("../data/games_mongodump.csv")
df.head(5)

,date_time,home_team,away_team,odd_home,odd_away,score_home,score_away,overtime,rest_time_home,rest_time_away
0,2015-03-13T01:00:00.000Z,Washington Wizards,Memphis Grizzlies,1.259740,4.030000,107,87,False,259200000,84600000
1,2015-03-12T04:30:00.000Z,Golden State Warriors,Detroit Pistons,1.050000,11.520000,105,98,False,174600000,86400000
2,2015-03-12T04:30:00.000Z,Portland Trail Blazers,Houston Rockets,1.729927,2.160000,105,100,False,351000000,347400000
3,2015-03-12T04:00:00.000Z,Phoenix Suns,Minnesota Timberwolves,1.490196,2.720000,106,97,False,172800000,171000000
4,2015-03-12T03:00:00.000Z,Denver Nuggets,Atlanta Hawks,3.450000,1.330033,115,102,False,172800000,178200000


In [7]:
# Label data
y = pd.DataFrame(df["score_home"]>df["score_away"],index=np.arange(len(df["score_home"])),columns=["home_wins"])
X = df.join(y)
X.head()

,date_time,home_team,away_team,odd_home,odd_away,score_home,score_away,overtime,rest_time_home,rest_time_away,home_wins
0,2015-03-13T01:00:00.000Z,Washington Wizards,Memphis Grizzlies,1.259740,4.030000,107,87,False,259200000,84600000,True
1,2015-03-12T04:30:00.000Z,Golden State Warriors,Detroit Pistons,1.050000,11.520000,105,98,False,174600000,86400000,True
2,2015-03-12T04:30:00.000Z,Portland Trail Blazers,Houston Rockets,1.729927,2.160000,105,100,False,351000000,347400000,True
3,2015-03-12T04:00:00.000Z,Phoenix Suns,Minnesota Timberwolves,1.490196,2.720000,106,97,False,172800000,171000000,True
4,2015-03-12T03:00:00.000Z,Denver Nuggets,Atlanta Hawks,3.450000,1.330033,115,102,False,172800000,178200000,True


In [8]:
# Generate Train and Test set
X = X.reindex(np.random.permutation(X.index))
m = np.ceil(len(X)*0.7)
X_train = X.iloc[0:m]
X_test = X.iloc[m:len(X)]
X_train.head()

/home/lguitton/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: slice indexers when using iloc should be integers and not floating point
/home/lguitton/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: slice indexers when using iloc should be integers and not floating point


,date_time,home_team,away_team,odd_home,odd_away,score_home,score_away,overtime,rest_time_home,rest_time_away,home_wins
8279,2008-10-24T04:00:00.000Z,Sacramento Kings,Houston Rockets,2.34,1.600000,97,110,False,259200000,NaN,False
8014,2009-01-29T02:00:00.000Z,Indiana Pacers,Milwaukee Bucks,1.38,3.150000,107,99,False,685800000,766800000,True
7863,2009-02-21T03:30:00.000Z,Chicago Bulls,Denver Nuggets,2.11,1.750000,116,99,False,174600000,178200000,True
7506,2009-03-30T03:00:00.000Z,Sacramento Kings,Phoenix Suns,2.99,1.380228,126,118,False,169200000,86400000,True
3504,2012-12-27T03:00:00.000Z,Memphis Grizzlies,Philadelphia 76ers,1.21,4.690000,89,99,False,345600000,277200000,False


In [9]:
# Feature Scaling
def apply_feature_scaling(X_j):
    return (X_j - np.mean(X_j))/(np.max(X_j)-np.min(X_j))
def feature_scale(X,x_array):
    for x_i in x_array:
        X[x_i]=apply_feature_scaling(X[x_i]);

In [11]:
feature_scale(X_train,['odd_home','odd_away','rest_time_home','rest_time_away']);
X_train.head()

/home/lguitton/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,date_time,home_team,away_team,odd_home,odd_away,score_home,score_away,overtime,rest_time_home,rest_time_away,home_wins
8279,2008-10-24T04:00:00.000Z,Sacramento Kings,Houston Rockets,0.029174,-0.078319,97,110,False,-0.006568,NaN,False
8014,2009-01-29T02:00:00.000Z,Indiana Pacers,Milwaukee Bucks,-0.029976,-0.009154,107,99,False,0.013027,0.017427,True
7863,2009-02-21T03:30:00.000Z,Chicago Bulls,Denver Nuggets,0.015003,-0.071626,116,99,False,-0.010454,-0.009493,True
7506,2009-03-30T03:00:00.000Z,Sacramento Kings,Phoenix Suns,0.069223,-0.088126,126,118,False,-0.010702,-0.013692,True
3504,2012-12-27T03:00:00.000Z,Memphis Grizzlies,Philadelphia 76ers,-0.040450,0.059566,89,99,False,-0.002600,-0.004965,False


# Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [14]:
fit_features = ['odd_home','odd_away','rest_time_home', 'rest_time_away']
X = X_train_map[fit_features]
y = X_train_map['home_wins']
classifier.fit(X ,y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [15]:
X_test.head()

,date_time,home_team,away_team,odd_home,odd_away,score_home,score_away,overtime,rest_time_home,rest_time_away,home_wins
1345,2014-10-31T03:00:00.000Z,Minnesota Timberwolves,Detroit Pistons,1.46,2.800000,97,91,False,90000000,86400000,True
1089,2014-11-28T20:00:00.000Z,Boston Celtics,Chicago Bulls,2.38,1.610000,102,109,False,414000000,230400000,False
4459,2012-02-20T03:00:00.000Z,Oklahoma City Thunder,Denver Nuggets,1.24,4.220000,124,118,True,172800000,172800000,True
67,2015-04-02T01:30:00.000Z,Detroit Pistons,Atlanta Hawks,2.12,1.757576,105,95,False,264600000,86400000,True
6216,2010-10-17T02:00:00.000Z,Memphis Grizzlies,Milwaukee Bucks,1.38,3.010000,91,77,False,345600000,90000000,True


In [ ]:
X = X_test_map[fit_features]
y = X_test_map['home_wins']
classifier.score(X,y)